In [2]:
from bs4 import BeautifulSoup
import requests as rt
import pandas as pd

In [3]:
url = 'https://josaa.admissions.nic.in/applicant/seatmatrix/OpeningClosingRankArchieve.aspx'
params = {
    #"ctl00$ContentPlaceHolder1$ddlroundno": 
    "ctl00$ContentPlaceHolder1$ddlInstype": "ALL",
    "ctl00$ContentPlaceHolder1$ddlInstitute": "ALL",
    "ctl00$ContentPlaceHolder1$ddlBranch": "ALL",
    "ctl00$ContentPlaceHolder1$ddlSeatType": "ALL",
    "ctl00$ContentPlaceHolder1$btnSubmit": "Submit"
}

In [4]:
years = ['2016', '2017', '2018', '2019', '2020', '2021', '2022']
rounds = ['1', '2', '3', '4', '5', '6', '7']

In [5]:
def scrape(Year, Round):   
    with rt.Session() as s:
        R = s.get(url)
        data = {}

        data.update({tag['name']: tag['value'] for tag in BeautifulSoup(R.content, 'lxml').select('input[name^=__]')})
        data["ctl00$ContentPlaceHolder1$ddlYear"] = Year
        R = s.post(url, data=data)

        data.update({tag['name']: tag['value'] for tag in BeautifulSoup(R.content, 'lxml').select('input[name^=__]')})
        data["ctl00$ContentPlaceHolder1$ddlroundno"] = Round
        R = s.post(url, data=data)

        for key, value in params.items():
            data.update({tag['name']: tag['value'] for tag in BeautifulSoup(R.content, 'lxml').select('input[name^=__]')})
            data[key] = value
            R = s.post(url, data=data)
        #print(R.text)
        #print(data)

    table = BeautifulSoup(R.text, 'lxml').find(id = 'ctl00_ContentPlaceHolder1_GridView1')
    df = pd.read_html(table.prettify())[0]
    df.dropna(inplace = True, how="all")
    
    df['Year'] = Year
    df['Round'] = Round
    #df['Opening Rank'] = df['Opening Rank'].astype(int)
    #df['Closing Rank'] = df['Closing Rank'].astype(int)
    
    return df

In [6]:
df = scrape('2022', '6')
df

KeyboardInterrupt: 

In [ ]:
#df['Opening Rank']= df['Opening Rank'].str.replace('P', '')
open = df.loc[(df['Seat Type']=='OPEN')& (df['Gender']=='Gender-Neutral') & (df.Quota=='AI')]
open['Opening Rank'] = open['Opening Rank'].astype(int)
open['Closing Rank'] = open['Closing Rank'].astype(int)
open

In [ ]:
open = open.sort_values('Opening Rank')
open.head(40)

In [ ]:
df['Academic Program Name'].str.split('Planning ')

In [ ]:
df.columns

In [ ]:
df.Quota.unique()

In [ ]:
df['Seat Type'].unique()

In [ ]:
df['Gender'].unique()

In [ ]:
df.to_csv('2022_all_6.csv')

In [ ]:
df['Institute'].unique()

In [ ]:
df['Opening Rank']

In [61]:
df = pd.read_csv('combinedv2.csv', low_memory=False)

In [62]:
df = df.drop(['Unnamed: 0'], axis = 1)

In [63]:
df['Type'] = 4
df

,Institute,Academic Program Name,Quota,Seat Type,Gender,Opening Rank,Closing Rank,Year,Round,Type
0,IIT Bhubaneswar,Civil Engineering (4 Years Bachelor of Technol...,AI,OPEN,Gender-Neutral,9193,11771,2022,1,4
1,IIT Bhubaneswar,Civil Engineering (4 Years Bachelor of Technol...,AI,OPEN,Female-only (including Supernumerary),16138,20164,2022,1,4
2,IIT Bhubaneswar,Civil Engineering (4 Years Bachelor of Technol...,AI,EWS,Gender-Neutral,1605,1744,2022,1,4
3,IIT Bhubaneswar,Civil Engineering (4 Years Bachelor of Technol...,AI,EWS,Female-only (including Supernumerary),3159,3159,2022,1,4
4,IIT Bhubaneswar,Civil Engineering (4 Years Bachelor of Technol...,AI,OBC-NCL,Gender-Neutral,3997,4297,2022,1,4
...,...,...,...,...,...,...,...,...,...,...
331427,Indian Institute of Crop Processing Technology...,Food Process Engineering (4 Years Bachelor of ...,AI,OPEN,NaN,10045.0,49647.0,2016,6,4
331428,Indian Institute of Crop Processing Technology...,Food Process Engineering (4 Years Bachelor of ...,AI,OPEN (PwD),NaN,1527.0,1895.0,2016,6,4
331429,Indian Institute of Crop Processing Technology...,Food Process Engineering (4 Years Bachelor of ...,AI,OBC-NCL,NaN,14330.0,16521.0,2016,6,4
331430,Indian Institute of Crop Processing Technology...,Food Process Engineering (4 Years Bachelor of ...,AI,SC,NaN,4650.0,8565.0,2016,6,4


In [64]:
df.loc[df['Institute'].str.contains('IIIT'),'Type' ] = 3

In [65]:
df[df['Institute'].str.contains('IIIT')]

,Institute,Academic Program Name,Quota,Seat Type,Gender,Opening Rank,Closing Rank,Year,Round,Type
8393,Atal Bihari Vajpayee IIIT & Management Gwalior,Computer Science and Engineering (4 Years Bach...,AI,OPEN,Gender-Neutral,4806,5932,2022,1,3
8394,Atal Bihari Vajpayee IIIT & Management Gwalior,Computer Science and Engineering (4 Years Bach...,AI,OPEN,Female-only (including Supernumerary),8385,11517,2022,1,3
8395,Atal Bihari Vajpayee IIIT & Management Gwalior,Computer Science and Engineering (4 Years Bach...,AI,OPEN (PwD),Gender-Neutral,185,185,2022,1,3
8396,Atal Bihari Vajpayee IIIT & Management Gwalior,Computer Science and Engineering (4 Years Bach...,AI,EWS,Gender-Neutral,768,894,2022,1,3
8397,Atal Bihari Vajpayee IIIT & Management Gwalior,Computer Science and Engineering (4 Years Bach...,AI,EWS,Female-only (including Supernumerary),1753,1753,2022,1,3
...,...,...,...,...,...,...,...,...,...,...
331101,IIIT (IIIT) Pune,Electronics and Communication Engineering (4 Y...,AI,OPEN (PwD),NaN,412.0,412.0,2016,6,3
331102,IIIT (IIIT) Pune,Electronics and Communication Engineering (4 Y...,AI,OBC-NCL,NaN,7204.0,11118.0,2016,6,3
331103,IIIT (IIIT) Pune,Electronics and Communication Engineering (4 Y...,AI,OBC-NCL (PwD),NaN,907.0,907.0,2016,6,3
331104,IIIT (IIIT) Pune,Electronics and Communication Engineering (4 Y...,AI,SC,NaN,3958.0,6471.0,2016,6,3


In [66]:
df.loc[df['Institute'].str.contains('NIT'),'Type' ] = 2
df

,Institute,Academic Program Name,Quota,Seat Type,Gender,Opening Rank,Closing Rank,Year,Round,Type
0,IIT Bhubaneswar,Civil Engineering (4 Years Bachelor of Technol...,AI,OPEN,Gender-Neutral,9193,11771,2022,1,4
1,IIT Bhubaneswar,Civil Engineering (4 Years Bachelor of Technol...,AI,OPEN,Female-only (including Supernumerary),16138,20164,2022,1,4
2,IIT Bhubaneswar,Civil Engineering (4 Years Bachelor of Technol...,AI,EWS,Gender-Neutral,1605,1744,2022,1,4
3,IIT Bhubaneswar,Civil Engineering (4 Years Bachelor of Technol...,AI,EWS,Female-only (including Supernumerary),3159,3159,2022,1,4
4,IIT Bhubaneswar,Civil Engineering (4 Years Bachelor of Technol...,AI,OBC-NCL,Gender-Neutral,3997,4297,2022,1,4
...,...,...,...,...,...,...,...,...,...,...
331427,Indian Institute of Crop Processing Technology...,Food Process Engineering (4 Years Bachelor of ...,AI,OPEN,NaN,10045.0,49647.0,2016,6,4
331428,Indian Institute of Crop Processing Technology...,Food Process Engineering (4 Years Bachelor of ...,AI,OPEN (PwD),NaN,1527.0,1895.0,2016,6,4
331429,Indian Institute of Crop Processing Technology...,Food Process Engineering (4 Years Bachelor of ...,AI,OBC-NCL,NaN,14330.0,16521.0,2016,6,4
331430,Indian Institute of Crop Processing Technology...,Food Process Engineering (4 Years Bachelor of ...,AI,SC,NaN,4650.0,8565.0,2016,6,4


In [67]:
df.loc[(df['Institute'].str.contains('IIT') )& (df['Type']==4),'Type' ] = 1
df

,Institute,Academic Program Name,Quota,Seat Type,Gender,Opening Rank,Closing Rank,Year,Round,Type
0,IIT Bhubaneswar,Civil Engineering (4 Years Bachelor of Technol...,AI,OPEN,Gender-Neutral,9193,11771,2022,1,1
1,IIT Bhubaneswar,Civil Engineering (4 Years Bachelor of Technol...,AI,OPEN,Female-only (including Supernumerary),16138,20164,2022,1,1
2,IIT Bhubaneswar,Civil Engineering (4 Years Bachelor of Technol...,AI,EWS,Gender-Neutral,1605,1744,2022,1,1
3,IIT Bhubaneswar,Civil Engineering (4 Years Bachelor of Technol...,AI,EWS,Female-only (including Supernumerary),3159,3159,2022,1,1
4,IIT Bhubaneswar,Civil Engineering (4 Years Bachelor of Technol...,AI,OBC-NCL,Gender-Neutral,3997,4297,2022,1,1
...,...,...,...,...,...,...,...,...,...,...
331427,Indian Institute of Crop Processing Technology...,Food Process Engineering (4 Years Bachelor of ...,AI,OPEN,NaN,10045.0,49647.0,2016,6,4
331428,Indian Institute of Crop Processing Technology...,Food Process Engineering (4 Years Bachelor of ...,AI,OPEN (PwD),NaN,1527.0,1895.0,2016,6,4
331429,Indian Institute of Crop Processing Technology...,Food Process Engineering (4 Years Bachelor of ...,AI,OBC-NCL,NaN,14330.0,16521.0,2016,6,4
331430,Indian Institute of Crop Processing Technology...,Food Process Engineering (4 Years Bachelor of ...,AI,SC,NaN,4650.0,8565.0,2016,6,4


In [68]:
pdf = df[df['Opening Rank'].str.contains('P')]
pdf

,Institute,Academic Program Name,Quota,Seat Type,Gender,Opening Rank,Closing Rank,Year,Round,Type
73,IIT Bhubaneswar,Electronics and Communication Engineering (4 Y...,AI,EWS (PwD),Gender-Neutral,11P,11P,2022,1,1
82,IIT Bhubaneswar,Mechanical Engineering (4 Years Bachelor of Te...,AI,OPEN (PwD),Gender-Neutral,68P,68P,2022,1,1
139,IIT Bombay,Aerospace Engineering (4 Years Bachelor of Tec...,AI,OBC-NCL (PwD),Gender-Neutral,14P,14P,2022,1,1
142,IIT Bombay,Aerospace Engineering (4 Years Bachelor of Tec...,AI,SC (PwD),Gender-Neutral,8P,8P,2022,1,1
160,IIT Bombay,Chemical Engineering (4 Years Bachelor of Tech...,AI,OBC-NCL (PwD),Gender-Neutral,10P,13P,2022,1,1
...,...,...,...,...,...,...,...,...,...,...
221881,IIT Jammu,Materials Science and Engineering (4 Years Bac...,AI,SC,Female-only (including Supernumerary),210P,210P,2019,7,1
221883,IIT Jammu,Materials Science and Engineering (4 Years Bac...,AI,ST,Female-only (including Supernumerary),1269P,1269P,2019,7,1
221900,IIT Dharwad,Computer Science and Engineering (4 Years Bach...,AI,ST,Female-only (including Supernumerary),141P,141P,2019,7,1
221909,IIT Dharwad,Electrical Engineering (4 Years Bachelor of Te...,AI,ST,Female-only (including Supernumerary),555P,555P,2019,7,1


In [69]:
pdf['Seat Type'].unique()

array(['EWS (PwD)', 'OPEN (PwD)', 'OBC-NCL (PwD)', 'SC (PwD)', 'ST', 'SC',
       'ST (PwD)'], dtype=object)

In [70]:
df = df[~df['Opening Rank'].str.contains('P')]
df = df[~df['Closing Rank'].str.contains('P')]
df

,Institute,Academic Program Name,Quota,Seat Type,Gender,Opening Rank,Closing Rank,Year,Round,Type
0,IIT Bhubaneswar,Civil Engineering (4 Years Bachelor of Technol...,AI,OPEN,Gender-Neutral,9193,11771,2022,1,1
1,IIT Bhubaneswar,Civil Engineering (4 Years Bachelor of Technol...,AI,OPEN,Female-only (including Supernumerary),16138,20164,2022,1,1
2,IIT Bhubaneswar,Civil Engineering (4 Years Bachelor of Technol...,AI,EWS,Gender-Neutral,1605,1744,2022,1,1
3,IIT Bhubaneswar,Civil Engineering (4 Years Bachelor of Technol...,AI,EWS,Female-only (including Supernumerary),3159,3159,2022,1,1
4,IIT Bhubaneswar,Civil Engineering (4 Years Bachelor of Technol...,AI,OBC-NCL,Gender-Neutral,3997,4297,2022,1,1
...,...,...,...,...,...,...,...,...,...,...
331427,Indian Institute of Crop Processing Technology...,Food Process Engineering (4 Years Bachelor of ...,AI,OPEN,NaN,10045.0,49647.0,2016,6,4
331428,Indian Institute of Crop Processing Technology...,Food Process Engineering (4 Years Bachelor of ...,AI,OPEN (PwD),NaN,1527.0,1895.0,2016,6,4
331429,Indian Institute of Crop Processing Technology...,Food Process Engineering (4 Years Bachelor of ...,AI,OBC-NCL,NaN,14330.0,16521.0,2016,6,4
331430,Indian Institute of Crop Processing Technology...,Food Process Engineering (4 Years Bachelor of ...,AI,SC,NaN,4650.0,8565.0,2016,6,4


In [71]:
import numpy as np
df['Opening Rank'] = df['Opening Rank'].astype(float)
df['Closing Rank'] = df['Closing Rank'].astype(float)
df['Opening Rank'] = np.floor(df['Opening Rank'])
df['Closing Rank'] = np.floor(df['Closing Rank'])
df['Opening Rank'] = df['Opening Rank'].astype(int)
df['Closing Rank'] = df['Closing Rank'].astype(int)
df

,Institute,Academic Program Name,Quota,Seat Type,Gender,Opening Rank,Closing Rank,Year,Round,Type
0,IIT Bhubaneswar,Civil Engineering (4 Years Bachelor of Technol...,AI,OPEN,Gender-Neutral,9193,11771,2022,1,1
1,IIT Bhubaneswar,Civil Engineering (4 Years Bachelor of Technol...,AI,OPEN,Female-only (including Supernumerary),16138,20164,2022,1,1
2,IIT Bhubaneswar,Civil Engineering (4 Years Bachelor of Technol...,AI,EWS,Gender-Neutral,1605,1744,2022,1,1
3,IIT Bhubaneswar,Civil Engineering (4 Years Bachelor of Technol...,AI,EWS,Female-only (including Supernumerary),3159,3159,2022,1,1
4,IIT Bhubaneswar,Civil Engineering (4 Years Bachelor of Technol...,AI,OBC-NCL,Gender-Neutral,3997,4297,2022,1,1
...,...,...,...,...,...,...,...,...,...,...
331427,Indian Institute of Crop Processing Technology...,Food Process Engineering (4 Years Bachelor of ...,AI,OPEN,NaN,10045,49647,2016,6,4
331428,Indian Institute of Crop Processing Technology...,Food Process Engineering (4 Years Bachelor of ...,AI,OPEN (PwD),NaN,1527,1895,2016,6,4
331429,Indian Institute of Crop Processing Technology...,Food Process Engineering (4 Years Bachelor of ...,AI,OBC-NCL,NaN,14330,16521,2016,6,4
331430,Indian Institute of Crop Processing Technology...,Food Process Engineering (4 Years Bachelor of ...,AI,SC,NaN,4650,8565,2016,6,4


In [73]:
df['Closing Rank'].dtype

dtype('int64')

In [74]:
df.to_csv('combinedv3.csv')

In [75]:
df[(df['Opening Rank']<30)&(df['Type']==2)]

,Institute,Academic Program Name,Quota,Seat Type,Gender,Opening Rank,Closing Rank,Year,Round,Type
3065,Malaviya NIT Jaipur,Architecture (5 Years Bachelor of Architecture),OS,OPEN (PwD),Gender-Neutral,28,28,2022,1,2
3126,Malaviya NIT Jaipur,Computer Science and Engineering (4 Years Bach...,HS,EWS (PwD),Gender-Neutral,21,21,2022,1,2
3145,Malaviya NIT Jaipur,Computer Science and Engineering (4 Years Bach...,OS,SC (PwD),Gender-Neutral,28,28,2022,1,2
3186,Malaviya NIT Jaipur,Electronics and Communication Engineering (4 Y...,HS,ST (PwD),Gender-Neutral,22,22,2022,1,2
3342,Maulana Azad NIT Bhopal,Computer Science and Engineering (4 Years Bach...,OS,ST (PwD),Gender-Neutral,13,13,2022,1,2
...,...,...,...,...,...,...,...,...,...,...
330424,NIT Warangal,Computer Science and Engineering (4 Years Bach...,OS,ST,NaN,14,183,2016,6,2
330443,NIT Warangal,Electronics and Communication Engineering (4 Y...,AP,SC (PwD),NaN,16,16,2016,6,2
330455,NIT Warangal,Electronics and Communication Engineering (4 Y...,OS,ST (PwD),NaN,17,17,2016,6,2
330579,Visvesvaraya NIT Nagpur,Architecture (5 Years Bachelor of Architecture),OS,SC (PwD),NaN,14,14,2016,6,2
